# Exploring cities around Europe 

## 1. Introduction

### 1.1 Description 

As everybody else, at some point, I started thinking about cities where I would enjoy living in. In my case, those thoughts came to my mind after I finished my Computer Engineering studies in Alicante (a beautiful and sunny city in the south-east of Spain). Thanks to the many posibilities for exchanges and internships that the University of Alicante gave me, I have gained some ideas about what is important to think about when you are planning about moving to a new city. 

In this project, we will take into account only countries in the European Union. The EU is a unique economic and political union between 28 EU countries that together cover much of the continent. It was created in 1958 and since then It has eliminated borders between the belonged countries, facilitating live, work and travelling abroad in Europe. Also, It created the Euro, a single currency which it mission is to stabilize the economy and help to get equality between the countries. 

But, of course, all these cities are very different between them, total population, culture, yearly sunny hours, language, average salary and working hours are some of the factors that completely change how is to live in a city. We will cluster by how similar they are in comparison to each other.

### 1.2 Data descrition

Classify a city can be very complicated and forces to get a lot of data from different sources:

- We will get the cities with it population and countries from wikipedia: https://en.wikipedia.org/wiki/List_of_cities_in_the_European_Union_by_population_within_city_limits

- From Foursquare API, we will extract relevant information about venues of each city.

- Openweathermap API will provide us with the historical information about the weather in each city

- To get the coordinates of each city, we will use opencagedata API.

### Future Improvements

- Add schengen zone
- Polution levels
- Number of job positions for xx studies
- Metropolitan area
- average salary
- average cost of living
- Transportation

In [1]:
from requests import get
import pandas as pd
from bs4 import BeautifulSoup

url = 'https://en.wikipedia.org/wiki/List_of_cities_in_the_European_Union_by_population_within_city_limits'
response = get(url)
html_soup = BeautifulSoup(response.text, 'html.parser')
table = html_soup.find('table', {'class': 'wikitable sortable'})
table_rows = table.find_all('tr')
table_columns = table.find_all('th')

bs4.element.ResultSet

In [10]:
c = []
for th in table_columns:
    text = th.text.replace('\n', '')
    c.append(text)

r = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text for tr in td]
    if(row == []):
        continue
    row = [element.strip('\n') for element in row]
    r.append(row)
len(r)

110

In [ ]:
#Get columns name
c = []
for th in table_columns:
    text = th.text.replace('\n', '')
    c.append(text)
    
#Get rows
r = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text for tr in td]
    if(row == []):
        continue
    row = [element.strip('\n') for element in row]
    r.append(row)

df = pd.DataFrame(r, columns=c)
df = df[df['Borough'] != 'Not assigned']
for i,row in df.iterrows():
    if (row['Neighbourhood'] == 'Not assigned'):
        row['Neighbourhood'] = row['Borough']
df_aux = df[['Postcode', 'Borough', 'Neighbourhood']]
df_aux = df_aux.groupby('Postcode')['Neighbourhood'].apply(','.join).reset_index()

df = pd.merge(df, df_aux, on=['Postcode'], how='inner')
df = df[['Postcode','Borough','Neighbourhood_y']]
df=df.rename(columns = {'Neighbourhood_y':'Neighbourhood'})
df = df.drop_duplicates(subset=['Postcode'], keep='first')

geo_data = pd.read_csv("https://cocl.us/Geospatial_data") 
geo_data=geo_data.rename(columns = {'Postal Code':'Postcode'})
df = pd.merge(df, geo_data, on=['Postcode'], how='inner')

print('There are {} Postal Codes.'.format(df.shape[0]))